In [1]:
import os
from environs import Env

import openai
import json
import pandas as pd

In [2]:
# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
env = Env()
env.read_env(".env") # read .env file, if it exists

api_key = os.getenv("OPENAI_API_KEY")

True

In [4]:
openai.api_key = 'api_key'

In [ ]:
df = pd.read_csv('../data/twitter_validation.csv', delimiter='\t')
df.head()

In [ ]:
df.head(50)